# Expense Claim Analysis

This notebook demonstrates how to create agents that use plugins to process travel expenses from local receipt images, generate an expense claim email, and visualize expense data using a pie chart. Agents dynamically choose functions based on the task context.

Steps:
1. OCR Agent processes the local receipt image and extracts travel expense data.
2. Email Agent generates an expense claim email.
3. Chart Agent visualizes expense data.

Created by:

<table>
<tr>
    <td align="center"><a href="https://github.com/ShivamGoyal03">
        <img src="https://github.com/ShivamGoyal03.png" width="100px;" alt="Shivam Goyal"/><br />
        <sub><b>Shivam Goyal</b></sub>
    </a><br />
    </td>
</tr></table>

Example of a travel expense scenario:
Imagine you're an employee traveling for a business meeting in another city. Your company has a policy to reimburse all reasonable travel-related expenses. Here’s a breakdown of potential travel expenses:
- Transportation:
Airfare for a round trip from your home city to the destination city.
Taxi or ride-hailing services to and from the airport.
Local transportation in the destination city (like public transit, rental cars, or taxis).

- Accommodation:
Hotel stay for three nights at a mid-range business hotel close to the meeting venue.

- Meals:
Daily meal allowance for breakfast, lunch, and dinner, based on the company's per diem policy.

- Miscellaneous Expenses:
Parking fees at the airport.
Internet access charges at the hotel.
Tips or small service charges.

- Documentation:
You submit all receipts (flights, taxis, hotel, meals, etc.) and a completed expense report for reimbursement.

## Install necessary packages

First, install the required Python packages: `pandas` and `plotly`.

In [ ]:
%pip install pandas plotly

## Import required libraries

Import the necessary libraries and modules for the notebook.

In [8]:
import os
from dotenv import load_dotenv
from azure.identity.aio import DefaultAzureCredential
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from semantic_kernel.kernel import Kernel
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.agents.azure_ai import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies import SequentialSelectionStrategy, DefaultTerminationStrategy
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.functions import kernel_function
import plotly.express as px
import pandas as pd
from pydantic import BaseModel, Field
from typing import List
from azure.ai.inference.models import SystemMessage, UserMessage, TextContentItem, ImageContentItem, ImageUrl, ImageDetailLevel

In [9]:
load_dotenv()
endpoint = "https://models.inference.ai.azure.com/"
token = os.environ.get("GITHUB_TOKEN")
model_name = "Phi-4-multimodal-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

 ## Define Expense Models

 Create a Pydantic model for individual expenses and an ExpenseFormatter class to convert a user query into structured expense data.

 Each expense will be represented in the format:
 `{'date': '07-Mar-2025', 'description': 'flight to destination', 'amount': 675.99, 'category': 'Transportation'}`


In [10]:
class Expense(BaseModel):
    date: str = Field(..., description="Date of expense in dd-MMM-yyyy format")
    description: str = Field(..., description="Expense description")
    amount: float = Field(..., description="Expense amount")
    category: str = Field(..., description="Expense category (e.g., Transportation, Meals, Accommodation, Miscellaneous)")

class ExpenseFormatter(BaseModel):
    raw_query: str = Field(..., description="Raw query input containing expense details")
    
    def parse_expenses(self) -> List[Expense]:
        """
        Parses the raw query and converts it into a list of Expense objects.
        Expected format for each expense: "date|description|amount|category"
        Expenses should be separated by semicolons.
        
        Example:
        "07-Mar-2025|flight to destination|675.99|Transportation; 07-Mar-2025|taxi from airport to hotel|24.00|Transportation"
        """
        expense_list = []
        for expense_str in self.raw_query.split(";"):
            if expense_str.strip():
                parts = expense_str.strip().split("|")
                if len(parts) == 4:
                    date, description, amount, category = parts
                    expense = Expense(
                        date=date.strip(),
                        description=description.strip(),
                        amount=float(amount.strip()),
                        category=category.strip()
                    )
                    expense_list.append(expense)
        return expense_list

## Defining Agents - Generating the Email

Create an agent class to generate an email for submitting an expense claim.
- This agent uses the `kernel_function` decorator to define a function that generates an email for submitting an expense claim.
- It calculates the total amount of the expenses and formats the details into an email body.

In [12]:
class ExpenseEmailAgent:

    @kernel_function(description="Generate an email to submit an expense claim to the Finance Team")
    async def generate_expense_email(expenses):
        total_amount = sum(expense['amount'] for expense in expenses)
        email_body = "Dear Finance Team,\n\n"
        email_body += "Please find below the details of my expense claim:\n\n"
        for expense in expenses:
            email_body += f"- {expense['description']}: ${expense['amount']}\n"
        email_body += f"\nTotal Amount: ${total_amount}\n\n"
        email_body += "Receipts for all expenses are attached for your reference.\n\n"
        email_body += "Thank you,\n[Your Name]"
        return email_body

# Agent for Extracting Travel Expenses from Receipt Images

Create an agent class to extract travel expenses from receipt images.
- This agent uses the `kernel_function` decorator to define a function that extracts travel expenses from receipt images.
- Convert the receipt image to text using OCR (Optical Character Recognition) and extract relevant information such as date, description, amount, and category.

In [11]:
class OCRAgentPlugin:
    def __init__(self):
        self.client = ChatCompletionsClient(
            endpoint="https://models.inference.ai.azure.com/",
            credential=AzureKeyCredential(os.environ.get("GITHUB_TOKEN")),
        )
        self.model_name = "Phi-4-multimodal-instruct"

    @kernel_function(description="Extract text from a local image using Phi-4 vision model")
    def extract_text(self, image_path: str) -> str:
        image_url_str = str(ImageUrl.load(image_file=image_path, image_format="jpg", detail=ImageDetailLevel.LOW))
        
        response = self.client.complete(
            messages=[
                SystemMessage(content="You are an OCR assistant, helping me extract text from images of receipts for travel expenses."),
                UserMessage(content=[
                    TextContentItem(text="Please extract the raw text from this receipt image, focusing on travel expenses like dates, descriptions, amounts, and categories (e.g., Transportation, Accommodation, Meals, Miscellaneous)."),
                    ImageContentItem(image_url=ImageUrl(url=image_url_str))
                ])
            ],
            model=self.model_name,
            temperature=0.1,
            max_tokens=2048
        )
        return response.choices[0].message.content

### Agent for Generating the Chart

Create an agent class to generate a pie chart using Plotly to visualize the distribution of expenses by category.
- This agent uses the `kernel_function` decorator to define a function that creates a pie chart using Plotly.
- It converts the expenses data into a Pandas DataFrame and generates a pie chart showing the distribution of expenses by category.


In [13]:
class ExpenseChartAgent:
    
    @kernel_function(description="Create a pie chart to visualize the distribution of expenses by category")
    def create_expense_chart(expenses):
        df = pd.DataFrame(expenses)
        fig = px.pie(df, values='amount', names='category', title='Expense Distribution by Category')
        return fig

## Processing Expenses

Define an asynchronous function to process the expenses by creating and registering the necessary agents and then invoking them.
- This function processes the expenses by loading environment variables, creating the necessary agents, and registering them as plugins.
- It creates a group chat with the two agents and sends a prompt message to generate the email and pie chart based on the expenses data.
- It handles any errors that occur during the chat invocation and ensures proper cleanup of the agents.

In [16]:
async def process_expenses():
    load_dotenv()
    ai_agent_settings = AzureAIAgentSettings.create()
    kernel = Kernel()

    # Get local image input or use sample data
    image_path = input("Enter the local file path for the receipt image (e.g., ./receipt.jpg): ").strip()
    if not image_path:
        print("No image provided. Using sample data instead.")
        user_query = (
            "07-Mar-2025|flight to destination|675.99|Transportation; "
            "07-Mar-2025|taxi from airport to hotel|24.00|Transportation; "
            "07-Mar-2025|dinner|65.50|Meals; "
            "07-Mar-2025|hotel stay|125.90|Accommodation"
        )
        formatter = ExpenseFormatter(raw_query=user_query)
        expenses = [expense.model_dump() for expense in formatter.parse_expenses()]
        initial_prompt = f"Process these travel expenses to generate an expense claim email and create a visualization: {str(expenses)}"
    else:
        initial_prompt = f"Process this local receipt image to extract travel expenses, generate an expense claim email, and create a visualization. Image path: {image_path}"

    # Connect to Azure AI
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as project_client,
    ):
        # Define agents with instructions
        ocr_agent_def = await project_client.agents.create_agent(
            model=ai_agent_settings.model_deployment_name,
            name="ocr_agent",
            instructions="Process the input to extract travel expense data from a local receipt image if an image path is provided in the prompt, then pass the extracted data forward for further processing. Use the 'extract_text' function from the 'ocrAgent' plugin when an image path is present."
        )
        ocr_agent = AzureAIAgent(client=project_client, definition=ocr_agent_def)

        email_agent_def = await project_client.agents.create_agent(
            model=ai_agent_settings.model_deployment_name,
            name="email_agent",
            instructions="Take the travel expense data from the previous agent and generate a professional expense claim email using the 'generate_expense_email' function from the 'expenseEmailAgent' plugin, then pass the data forward."
        )
        email_agent = AzureAIAgent(client=project_client, definition=email_agent_def)

        chart_agent_def = await project_client.agents.create_agent(
            model=ai_agent_settings.model_deployment_name,
            name="chart_agent",
            instructions="Take the travel expense data from the previous agent and create a pie chart to visualize the distribution by category using the 'create_expense_chart' function from the 'expenseChartAgent' plugin."
        )
        chart_agent = AzureAIAgent(client=project_client, definition=chart_agent_def)

        # Register plugins with the kernel
        kernel.add_plugin(OCRAgentPlugin(), plugin_name="ocrAgent")
        kernel.add_plugin(ExpenseEmailAgent(), plugin_name="expenseEmailAgent")
        kernel.add_plugin(ExpenseChartAgent(), plugin_name="expenseChartAgent")

        # Create group chat
        chat = AgentGroupChat(
            agents=[ocr_agent, email_agent, chart_agent],
            selection_strategy=SequentialSelectionStrategy(initial_agent=ocr_agent),
            termination_strategy=DefaultTerminationStrategy(maximum_iterations=3)
        )

        # Add user message with prompt
        user_message = ChatMessageContent(
            role=AuthorRole.USER,
            content=initial_prompt
        )
        
        await chat.add_chat_message(user_message)
        print(f"# AuthorRole.USER: \n{user_message.content}\n")

        # Invoke chat and let agents process autonomously
        try:
            async for content in chat.invoke():
                print(f"# {content.role} - {content.name}: \n{content.content}\n")
        except Exception as e:
            print(f"Error during chat invocation: {e}")
        finally:
            await chat.reset()
            await project_client.agents.delete_agent(ocr_agent.id)
            await project_client.agents.delete_agent(email_agent.id)
            await project_client.agents.delete_agent(chart_agent.id)

## Main function

Define the main function to clear the console and run the `process_expenses` function asynchronously.

In [17]:
async def main():
    # Clear the console
    os.system('cls' if os.name=='nt' else 'clear')

    # Run the async agent code
    await process_expenses()

await main()

# AuthorRole.USER: 
Process this local receipt image to extract travel expenses, generate an expense claim email, and create a visualization. Image path: ./receipt.jpg

# AuthorRole.ASSISTANT - ocr_agent: 
It seems there's an issue with extracting the text from the local receipt image. Could you provide the details manually, or alternatively, upload the image to a cloud service where it can be accessed via a URL?

# AuthorRole.ASSISTANT - email_agent: 
It seems there's an issue with extracting the text from the local receipt image. Could you provide the details manually, or alternatively, upload the image to a cloud service where it can be accessed via a URL?

# AuthorRole.ASSISTANT - chart_agent: 
It seems you're asking to process an image for travel expense details, generate an expense claim email, and create a visualization. However, I cannot access local files directly or process image data. Could you provide the expense details manually, or upload the image to a cloud service wher